## Case study: modelling impact of climate risk on cement plants

In [ ]:
# pip install nbformat pandas plotly requests
# import nest_asyncio
# nest_asyncio.apply()

In [ ]:
import json

import plotly.io

plotly.io.renderers.default = "notebook"

from dependency_injector import providers
from physrisk.container import Container
from physrisk.vulnerability_models.vulnerability import VulnerabilityModelsFactory
from physrisk.vulnerability_models.config_based_impact_curves import (
    config_items_from_csv,
)

Define a portfolio. The "type" attribute can be used to define an arbitrary scheme to capture categories of assets, matching the vulnerability configuration. 

In [2]:
portfolio = {
    "items": [
        {
            "asset_class": "ManufacturingAsset",
            "type": "CementPlant",
            "latitude": 24.0426,
            "longitude": 91.0158,
        },
        {
            "asset_class": "ManufacturingAsset",
            "type": "CementPlant",
            "latitude": 22.6588,
            "longitude": 90.3373,
        },
    ]
}
request = {
    "assets": portfolio,
    "include_asset_level": True,
    "include_calc_details": True,
    "include_measures": False,
    "years": [2050],
    "scenario": "ssp585",
}

Configuration can be defined programmatically, but is also intended to be maintained in CSV format. Here we write the config line to CSV and read back again. In practise, a file of config lines can be maintained. 

In [3]:
config_items = config_items_from_csv("cement_plants_example_config.csv")

Having defined the configuration the calculation is run. Dependency injection is used to apply the custom configuration. 

In [5]:
# we make use of dependency injection to inject a vulnerability model that accepts the configuration.
container = Container()
container.override_providers(
    vulnerability_models_factory=providers.Factory(
        VulnerabilityModelsFactory, config=config_items
    )
)
requester = container.requester()
response = json.loads(
    requester.get(request_id="get_asset_impact", request_dict=request)
)